# Assignment 3

In [ ]:
# Data manipulation
import numpy as np
import pandas as pd
# Data visualization
# import bokeh.plotting as bp
# import bokeh.models as bm
# import bokeh.core.properties as bcp
# import bokeh.io as bio
import plotly as pl
import plotly.graph_objs as pgo
%pylab inline
# Interactive widget functionality
from IPython.display import display, HTML
from ipywidgets import widgets, Layout, Box, VBox

# Set paths
fig_dir = './figs/'
data_dir = './data/'

# Refresh data flag
refresh_data = False

# Display result in an iframe
def show_iframe(url, height=400, width=1000):
    display_string = '<iframe src={url} width={w} height={h}></iframe>'.format(url=url, w=width, h=height)
    print(display_string)
    return HTML(display_string)

# Import Data

In [9]:
! head -10 ./data/NJ_data.csv

County FIPS code	County name	State	Average out-of-pocket healthcare spending	Average out-of-pocket healthcare burden (%)	Average estimated annual gross income	Average age	Sample size	State Medicaid expansion status as of 2016 (1=Expanded; 0=Did not expand)	County uninsurance rate in 2016 (%)	Percent change in out-of-pocket healthcare spending (2015-2016)	Percentage point change of out-of-pocket healthcare burden (2015-2016)	Health Coutcomes/Z-Score	Health Factors/Z-Score	Length of Life/Z-Score	Quality of Life/Z-Score	Health Behaviors/Z-Score	Clinical Care/Z-Score	Social & Economic Factors/Z-Score	Physical Environment/Z-Score	Premature Death/Years of Potential Life Lost Rate	% Fair/Poor Health	Physically Unhealthy Days	Mentally Unhealthy Days	% Low Birth Weight	% Smokers (Adult)	% Obese (Adult)	Food Environment Index	% Physically Inactive	% With Access to Exercise Opportunities	% Excessive Drinking	% Uninsured	% Uninsured Adults	% Uninsured Children	Primary Care Physician-Resident Ratio

## Prepare Data

In [31]:
import seaborn as sns
sns.set()

# Load the example tips dataset
cty = pd.read_csv('./data/NJ_data.csv')

# Plot tip as a function of toal bill across days
g = sns.lmplot(x="%_Female", y="Average_out-of-pocket_healthcare_spending", hue="County_name", data=cty)

# Use more informative axis labels than are provided by default
g.set_axis_labels("Percent Female", "Average healthcare spending")

KeyError: "['%_Female' 'Average_out-of-pocket_healthcare_spending' 'County_name'] not in index"

In [19]:
# Plot tip as a function of toal bill across days
g = sns.lmplot(x="%_Female", y="Average_out-of-pocket_healthcare_spending", hue="County_name",
               data = cty)

# Use more informative axis labels than are provided by default
g.set_axis_labels("Percent Female", "Average healthcare spending")

KeyError: "['%_Female' 'Average_out-of-pocket_healthcare_spending' 'County_name'] not in index"

In [ ]:
sorted(set(cty['dimname'])

In [ ]:
# Read in data
cty = pd.read_csv('./data/NJ_data.csv')

In [ ]:
# Convert months to pandas periods
#cty['month'] = cty['periodid'].apply(lambda x: pd.Period(x, freq='M'))

# Map income categories to quintiles
#inc_dict = {
#    '<21': 'q1',
#    '21-40': 'q2',
#    '41-60': 'q3',
#    '61-80': 'q4',
#    '81-100': 'q5'
#}
#cty['category'] = cty['category'].replace(int_dict)

# Capture category labels for each dimension in the right order
#dim_labs = {
#    'age': ['<25', '25-34', '35-44', '45-54', '55-64', '65+'],
#    'income': ['q1', 'q2', 'q3', 'q4', 'q5'],
#    'bizsize': ['SMALL', 'MEDIUM', 'LARGE'],
#    'location': ['Same Neighborhood', 'Same Region', 'Different Region'],
#    'product': ['Durables', 'Fuel', 'Nondurables', 'Other Services', 'Restaurants']
#}

# Construct lists of areas, dimensions and measures
areas = (cty['State'])
#dims = sorted(set(cty['dimname']))
measures = ['Average out-of-pocket healthcare spending']

# Provide integer representation of months
#months = sorted(set(cty['month']))
#months_int = range(len(months))
#months_dict = dict(zip(months, months_int))
#cty['month_int'] = cty['month'].map(months_dict)

# Define LCC palette
cty_colors = ['#00a0dd', '#a2dadb', '#bbd976', '#ffe18b', '#fbaf5d', '#f57f32']

cty.head()

With our data in hand, it would be nice to visualize the components that we want.  Before we get to interactivity, let's just create static views of the content first.  The first step is to be able to isolate the subset of the data we want.  In particular, we want to be able to capture the information for the subset that corresponds to a particular area, dimension, and growth measure.

In [ ]:
cty[cty['State'] == 'NJ']['Average out-of-pocket healthcare spending'].unstack('category')

In [ ]:
def lcc_sub(area, dim, measure, df=lcc):
    # Define area and dimension subset conditions
    a = (df['area_name'] == area)
    d = (df['dimname'] == dim)
    # Subset data
    df_sub = df[a & d][['month', 'category', measure]]
    # Set month and category to the index
    df_sub.set_index(['month', 'category'], inplace=True)
    # Unstack category (aka - convert from long to wide format)
    df_sub = df_sub.unstack('category') * 100
    # Drop multiindex columns in favor of category names
    df_sub.columns = [c[1] for c in df_sub.columns]
    # If we are look at anything but the total growth rate, fix label order
    if dim != 'Total':
        df_sub = df_sub[dim_labs[dim]]
    return df_sub.reset_index()

lcc_sub('Chicago, IL - Metro Area', 'age', 'growth_contribution').head()

Now we need to plot these data.  If we are looking at growth contributions, we want a stacked bar chart.  If we are looking at growth rates, we want a line chart.  So, we need to define two functions to handle these scenarios.

In [ ]:
# def plot_sub_bar(area, dim, measure, df=lcc, colors=lcc_colors, out_file='../figs/lcc_stacked_bar.html'):
#     # Define location of the output file
#     bp.output_file(out_file)
#     # Capture relevant subset (note how we embed our first function)
#     df_sub = lcc_sub(area, dim, measure, df)
#     # Cast subset as a ColumnDataSource
#     source = bm.ColumnDataSource(df_sub)
#     # Create figure
#     fig = bp.figure(x_range=(list(months_int)[0], list(months_int)[-1]), y_range=(-6, 6), plot_width=900, plot_height=500, 
#                     title='{m} by {d} in {a}'.format(m=measure, d=dim, a=area))
#     # Generate vertical stacked chart
#     fig.vbar_stack(dim_labs[dim], x='month_int', width=0.9, source=source, 
#                    color=colors, legend=[bcp.value(d) for d in dim_labs[dim]])
#     # Save file
#     bp.save(fig)
#     return out_file
    
# stack_bar = plot_sub_bar('Chicago, IL - Metro Area', 'age', 'growth_contribution')

# show_iframe(stack_bar, height=550, width=950)

In [ ]:
df_sub = lcc_sub('Chicago, IL - Metro Area', 'age', 'growth_contribution')
df_sub['month_str'] = df_sub['month'].apply(lambda x: x.strftime("%m-%Y"))
df_sub.head()

In [ ]:
for i,d in enumerate(dim_labs['age']):
    print(d)
    tmp_bar = pgo.Bar(x=df_sub['month_str'], y=df_sub[d], name=d, marker=dict(color=colors[i]))
    

In [ ]:
[l.upper() for l in 'abcde']

In [ ]:
def plot_sub_bar(area, dim, measure, df=lcc, colors=lcc_colors, out_file='../figs/lcc_stacked_bar.html'):
    # Capture relevant subset (note how we embed our first function)
    df_sub = lcc_sub(area, dim, measure, df)
    # Convert months to string
    df_sub['month_str'] = df_sub['month'].apply(lambda x: x.strftime("%m-%Y"))
    # Generate data traces for plotly
    data = [pgo.Bar(x=df_sub['month_str'], y=df_sub[d], name=d, marker=dict(color=colors[i])) for i,d in enumerate(dim_labs[dim])]
    # Define plotly layout
    layout = pgo.Layout(barmode='stack', title='{m} in {a} by {d}'.format(a=area, d=dim, m=measure),  yaxis=dict(title='Year-over-Year Growth Contributions (pp)'))
    # Define map figure
    stack_fig = {
        'data': data,
        'layout': layout
    }
    return stack_fig
    
stack_fig = plot_sub_bar('Chicago, IL - Metro Area', 'age', 'growth_contribution')

# Plot data
pl.plotly.iplot(stack_fig, filename='lcc_stacked_bar')

In [ ]:
def plot_sub_line(area, dim, measure, df=lcc, colors=lcc_colors, out_file='../figs/lcc_line.html'):
    # Capture relevant subset (note how we embed our first function)
    df_sub = lcc_sub(area, dim, measure, df)
    # Convert months to string
    df_sub['month_str'] = df_sub['month'].apply(lambda x: x.strftime("%m-%Y"))
    # Generate data traces for plotly
    data = [pgo.Scatter(x=df_sub['month_str'], y=df_sub[d], name=d, line=dict(color=colors[i])) for i,d in enumerate(dim_labs[dim])]
    # Define plotly layout
    layout = pgo.Layout(title='{m} in {a} by {d}'.format(a=area, d=dim, m=measure), yaxis=dict(title='Year-over-Year Growth Rate (%)'))
    # Define map figure
    line_fig = {
        'data': data,
        'layout': layout
    }
    return line_fig
    
line_fig = plot_sub_line('Chicago, IL - Metro Area', 'age', 'growth_contribution')

# Plot data
pl.plotly.iplot(line_fig, filename='lcc_line')

## Prepare Interactive Widgets

To enable interactive exploration of the data, we are going to employ [IPython widgets](https://ipywidgets.readthedocs.io/en/latest/). These allow the user to change the content of the desired chart.

In [ ]:
area_select = widgets.Dropdown(
                options = areas,
                value = 'National',
                description = 'Area:',
                disabled = False)

compare_select = widgets.SelectMultiple(
                    options = areas,
                    value = ['Atlanta, GA - Metro Area'],
                    description = 'Comparison Areas:',
                    disabled = False,
                    rows=5)

dim_select = widgets.Dropdown(
                options = dims,
                value = 'age',
                description = 'Dimension:',
                disabled = False)

meas_select = widgets.Dropdown(
                options = measures,
                value = 'growth_contribution',
                description = 'Measure:',
                disabled = False)


# Construct composite box to hold all selectors
selections = [area_select, dim_select, meas_select]

primary_layout = Layout(display='flex', flex_flow='columns', justify_content='flex-start', align_items='stretch')

primary_select_box = VBox(children=selections, layout=primary_layout)
 
secondary_layout = Layout(display='flex', flex_flow='columns', justify_content='flex-start',
                          align_items='stretch')

compare_select_box = Box(children=[compare_select],
                         layout=secondary_layout)

select_box = Box(children=[primary_select_box, compare_select_box], layout=primary_layout)

display(select_box)

In [ ]:
line_fig = plot_sub_line(area_select.value, dim_select.value, meas_select.value)

# Plot data
pl.plotly.iplot(line_fig, filename='lcc_line')